In [1]:
import findspark
findspark.init('/usr/hdp/current/spark2-client')
findspark.find()

'/usr/hdp/current/spark2-client'

In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("ch02Introduction").getOrCreate()

In [4]:
print(spark.version)

2.0.0.2.5.0.0-1245


In [5]:
sc = spark.sparkContext

In [16]:
sc._conf.getAll()

('spark.history.kerberos.keytab', 'none')

In [22]:
for x in sc._conf.getAll():
    if 'PROXY' in x[0]:
        print(x[1])

http://rm01.itversity.com:19288/proxy/application_1528589352821_34259
rm01.itversity.com


In [17]:
sc._conf.getAll()

[('spark.history.kerberos.keytab', 'none'),
 ('spark.dynamicAllocation.maxExecutors', '4'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.history.ui.port', '18081'),
 ('spark.driver.extraLibraryPath',
  '/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES',
  'http://rm01.itversity.com:19288/proxy/application_1528589352821_34259'),
 ('spark.yarn.historyServer.address', 'gw01.itversity.com:18081'),
 ('spark.executor.extraLibraryPath',
  '/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 ('spark.driver.port', '35058'),
 ('spark.history.provider',
  'org.apache.spark.deploy.history.FsHistoryProvider'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_HOSTS',
  'rm01.itversity.com'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.deployMode', 'client'),

In [18]:
sc._jsc.sc().uiWebUrl().get()

'http://172.16.1.109:4044'

In [23]:
myRange = spark.range(1000).toDF("number")

In [24]:
myRange.take(3)

[Row(number=0), Row(number=1), Row(number=2)]

In [25]:
divisBy2 = myRange.where("number %2 = 0")

In [26]:
divisBy2.show()

+------+
|number|
+------+
|     0|
|     2|
|     4|
|     6|
|     8|
|    10|
|    12|
|    14|
|    16|
|    18|
|    20|
|    22|
|    24|
|    26|
|    28|
|    30|
|    32|
|    34|
|    36|
|    38|
+------+
only showing top 20 rows



In [27]:
pwd

'/home/kranthidr/projects/pyspark-lab'

In [28]:
flightData2015 = spark\
                .read\
                .option("inferSchema", True)\
                .option("header", True)\
                .csv('/user/kranthidr/spark-guide/data/flight-data/csv/2015-summary.csv')

In [29]:
flightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [30]:
flightData2015.sort("count").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Estonia|    1|
|              Kosovo|      United States|    1|
|              Zambia|      United States|    1|
|       United States|   Papua New Guinea|    1|
|               Malta|      United States|    1|
|       United States|          Gibraltar|    1|
|            Suriname|      United States|    1|
|       United States|            Croatia|    1|
|            Djibouti|      United States|    1|
|        Burkina Faso|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|             Cyprus|    1|
|       United States|          Singapore|    1|
|             Moldova|      United States|    1|
|              Cyprus|      United States|    1|
|       United States|          Lithuania|    1|
|       United States|           Bulgaria|    1|
|       United State

In [31]:
flightData2015.sort("count").explain()

== Physical Plan ==
*Sort [count#12 ASC], true, 0
+- Exchange rangepartitioning(count#12 ASC, 200)
   +- *Scan csv [DEST_COUNTRY_NAME#10,ORIGIN_COUNTRY_NAME#11,count#12] Format: CSV, InputPaths: hdfs://nn01.itversity.com:8020/user/kranthidr/spark-guide/data/flight-data/csv/2015-summary.csv, PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>


In [32]:
spark.conf.set("spark.sql.shuffle.partitions", "5")
flightData2015.sort("count").take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1)]

In [37]:
spark.conf.get('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES')

'http://rm01.itversity.com:19288/proxy/application_1528589352821_34259'

In [40]:
spark.conf.get('spark.driver.appUIAddress')

'http://172.16.1.109:4044'

In [41]:
flightData2015.createOrReplaceTempView("flight_data_2015")

In [42]:
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, COUNT(1) 
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME""")


dataFrameWay = flightData2015.groupBy("DEST_COUNTRY_NAME").count()

In [43]:
sqlWay.explain()
dataFrameWay.explain()

== Physical Plan ==
*HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#10, 5)
   +- *HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[partial_count(1)])
      +- *Scan csv [DEST_COUNTRY_NAME#10] Format: CSV, InputPaths: hdfs://nn01.itversity.com:8020/user/kranthidr/spark-guide/data/flight-data/csv/2015-summary.csv, PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
== Physical Plan ==
*HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#10, 5)
   +- *HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[partial_count(1)])
      +- *Scan csv [DEST_COUNTRY_NAME#10] Format: CSV, InputPaths: hdfs://nn01.itversity.com:8020/user/kranthidr/spark-guide/data/flight-data/csv/2015-summary.csv, PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


In [44]:
sqlWay.show()
dataFrameWay.show()

+--------------------+--------+
|   DEST_COUNTRY_NAME|count(1)|
+--------------------+--------+
|             Moldova|       1|
|             Bolivia|       1|
|             Algeria|       1|
|Turks and Caicos ...|       1|
|            Pakistan|       1|
|    Marshall Islands|       1|
|            Suriname|       1|
|              Panama|       1|
|         New Zealand|       1|
|             Liberia|       1|
|             Ireland|       1|
|              Zambia|       1|
|            Malaysia|       1|
|               Japan|       1|
|    French Polynesia|       1|
|           Singapore|       1|
|             Denmark|       1|
|               Spain|       1|
|             Bermuda|       1|
|            Kiribati|       1|
+--------------------+--------+
only showing top 20 rows

+--------------------+-----+
|   DEST_COUNTRY_NAME|count|
+--------------------+-----+
|             Moldova|    1|
|             Bolivia|    1|
|             Algeria|    1|
|Turks and Caicos ...|    1|
|  

In [46]:
spark.sql("SELECT max(count) FROM flight_data_2015").show()

+----------+
|max(count)|
+----------+
|    370002|
+----------+



In [45]:
from pyspark.sql.functions import max
flightData2015.select(max("count")).show()

+----------+
|max(count)|
+----------+
|    370002|
+----------+



In [47]:
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, SUM(count) AS destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY destination_total DESC
LIMIT 5""")

from pyspark.sql.functions import desc
maxDataFrame = flightData2015.groupBy("DEST_COUNTRY_NAME").sum("count")\
.withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total"))\
.limit(5)

In [48]:
maxSql.explain()
maxDataFrame.explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#96L DESC], output=[DEST_COUNTRY_NAME#10,destination_total#96L])
+- *HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[sum(cast(count#12 as bigint))])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#10, 5)
      +- *HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[partial_sum(cast(count#12 as bigint))])
         +- *Scan csv [DEST_COUNTRY_NAME#10,count#12] Format: CSV, InputPaths: hdfs://nn01.itversity.com:8020/user/kranthidr/spark-guide/data/flight-data/csv/2015-summary.csv, PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>
== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#113L DESC], output=[DEST_COUNTRY_NAME#10,destination_total#113L])
+- *HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[sum(cast(count#12 as bigint))])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#10, 5)
      +- *HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[pa

In [49]:
maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [50]:
maxDataFrame.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+

